## Setup i konfiguracja

## Kontekst i wymagania

- **Dzień szkolenia**: Dzień 3 - Integracje i Governance
- **Typ notebooka**: Demo
- **Wymagania techniczne**:
  - Uprawnienia do tworzenia SQL Warehouses (lub dostęp do istniejącego)
  - Unity Catalog

## Wstęp teoretyczny

**Cel sekcji:** Zrozumienie jak udostępniać dane z Lakehouse do świata zewnętrznego i użytkowników biznesowych.

**Podstawowe pojęcia:**
- **SQL Warehouse**: Zoptymalizowany silnik obliczeniowy dla zapytań SQL (nie dla kodu Python/Scala).
- **Genie**: Inteligentny asystent danych, który rozumie strukturę tabel i odpowiada na pytania w języku naturalnym.
- **Direct Lake**: Tryb połączenia Power BI, który czyta pliki Parquet bezpośrednio, pomijając warstwę SQL (najszybszy).

## Izolacja per użytkownik

In [ ]:
%run ../00_setup

## Konfiguracja środowiska

In [ ]:
from pyspark.sql import functions as F

# Ustawienie katalogu i schematu
spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE SCHEMA {GOLD_SCHEMA}")

print(f"Working in: {CATALOG}.{GOLD_SCHEMA}")

## Sekcja 1: Databricks SQL Warehouses

SQL Warehouses to "serce" warstwy BI w Databricks.

### Typy Warehouse'ów

1.  **Serverless**: Startuje w sekundy, skaluje się automatycznie. Rekomendowany.
2.  **Pro**: Wykorzystuje silnik Photon, ale wymaga dłuższego startu (chyba że jest pula).
3.  **Classic**: Starsza architektura (VM-based).

In [ ]:
# Przykład definicji konfiguracji Warehouse (JSON)
# Można to wykorzystać w API Databricks do automatyzacji tworzenia

warehouse_config = {
    "name": "KION_BI_Warehouse_Demo",
    "cluster_size": "2X-Small",
    "min_num_clusters": 1,
    "max_num_clusters": 2,
    "auto_stop_mins": 10,
    "enable_serverless_compute": True,
    "warehouse_type": "PRO",
    "tags": {
        "department": "Sales",
        "cost_center": "1234"
    }
}

import json
print(json.dumps(warehouse_config, indent=2))

## Sekcja 2: Databricks Genie (AI/BI)

Genie pozwala na zadawanie pytań do danych bez znajomości SQL.

### Przygotowanie danych pod Genie

Aby Genie działało dobrze, musimy zadbać o metadane (komentarze do tabel i kolumn).

In [ ]:
# Dodawanie komentarzy do tabeli Gold (kluczowe dla Genie!)
spark.sql(f"""
COMMENT ON TABLE {CATALOG}.{GOLD_SCHEMA}.fact_sales IS 
'Tabela faktów zawierająca transakcje sprzedaży. Zawiera kwoty, daty i klucze obce do wymiarów.'
""")

spark.sql(f"""
COMMENT ON COLUMN {CATALOG}.{GOLD_SCHEMA}.fact_sales.total_amount IS 
'Całkowita wartość zamówienia w PLN (brutto).'
""")

print("Metadane zaktualizowane. Genie będzie teraz lepiej rozumieć te dane.")

## Sekcja 3: Integracje Zewnętrzne (Power BI & Dremio)

### Power BI - Direct Lake vs Direct Query

- **Direct Lake**: Power BI Service -> OneLake/Storage (Parquet). Wymaga Fabric lub odpowiedniej konfiguracji.
- **Direct Query**: Power BI -> SQL Warehouse -> Storage.

### Przygotowanie dedykowanego widoku

Dla narzędzi BI (Dremio, Power BI) dobrą praktyką jest tworzenie widoków, które ukrywają logikę złączeń.

In [ ]:
# Tworzenie widoku raportowego
view_name = "v_sales_summary_bi"

spark.sql(f"""
CREATE OR REPLACE VIEW {CATALOG}.{GOLD_SCHEMA}.{view_name} AS
SELECT 
    c.country,
    year(f.order_date) as year,
    month(f.order_date) as month,
    count(distinct f.order_id) as orders_count,
    sum(f.total_amount) as total_revenue
FROM {CATALOG}.{GOLD_SCHEMA}.fact_sales f
JOIN {CATALOG}.{GOLD_SCHEMA}.dim_customer c ON f.customer_id = c.customer_id
GROUP BY 1, 2, 3
""")

print(f"Widok {view_name} gotowy do podłączenia w BI.")

In [ ]:
# Weryfikacja widoku
display(spark.table(f"{CATALOG}.{GOLD_SCHEMA}.{view_name}"))

## Best Practices

### Wydajność BI:
- Używaj **Serverless SQL Warehouses** dla najlepszego UX (szybki start).
- Włącz **Photon** (domyślne w Serverless/Pro).
- Stosuj **Materialized Views** dla ciężkich agregacji, jeśli dashboardy działają wolno.

### Governance:
- Nie podłączaj BI bezpośrednio do tabel Silver/Bronze. Używaj tylko **Gold**.
- Używaj dedykowanych **Service Principals** do połączeń z BI, a nie kont osobistych.

## Podsumowanie

1.  Skonfigurowaliśmy metadane dla **Genie**.
2.  Omówiliśmy typy **SQL Warehouses**.
3.  Przygotowaliśmy zoptymalizowany widok dla **Power BI / Dremio**.

## Czyszczenie zasobów

In [ ]:
# spark.sql(f"DROP VIEW IF EXISTS {CATALOG}.{GOLD_SCHEMA}.{view_name}")
print("Zasoby zachowane do dalszych ćwiczeń.")